In [ ]:
import os, glob
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
pd.options.display.max_columns=None
pd.options.display.max_rows=None

In [ ]:
ibisdir = Path(os.environ['DATA'], 'Ben', 'IBIS', 'white_light')
calfile = str(ibisdir.joinpath('calibration_rs4', 'param_sweep_boxcar_fwhm37.csv'))
df = pd.read_csv(calfile)
print(df.keys()[14:25], df.keys()[25:36])
vdrift = np.arange(-1, 1.1, 0.2)
vtop4 = df.iloc[0,14:25]
vbot4 = df.iloc[0,25:36]

In [ ]:
calfile = str(ibisdir.joinpath('calibration_rs8', 'param_sweep_boxcar_fwhm37.csv'))
df = pd.read_csv(calfile)
print(df.keys()[14:25], df.keys()[25:36])
display(df)
vdrift = np.arange(-1, 1.1, 0.2)
vtop8 = df.iloc[0,14:25]
vbot8 = df.iloc[0,25:36]

In [ ]:
FS = 16
plt.figure(figsize=(10,10))
plt.plot(vdrift, vtop4.values, 'b+-', label='Rs = 4px')
plt.plot(vdrift, vtop8.values, 'co-', label='Rs = 8px')
plt.xlabel('Imposed drift (px/frame)', fontsize=FS)
plt.ylabel('Measured mean velocity (px/frame)', fontsize=FS)
plt.axis([-1, 1, -1, 1])
plt.grid()
plt.legend()

In [ ]:
calfiles = list(ibisdir.rglob('calibration_rs8_*/param_sweep_*.csv'))
calfiles

In [ ]:
# Concatenate all csv file content into one dataframe
df_list = [pd.read_csv(f) for f in calfiles]
df0 = df_list[0]
display(df0)
vtop8 = df0.iloc[:,14:25]
display(vtop8)

In [ ]:
vdrift = np.arange(-0.5, 0.51, 0.1)
df_list2 = []
for df in df_list:
    vtop = df.iloc[:, 14:25]
    vbot = df.iloc[:, 14:25]
    df2 = df.iloc[:,0:14].drop(columns=['index', 'fwhm']).loc[df.index.repeat(11)]
    df2['vx_drift'] = vdrift
    df2['vx_top'] = vtop.iloc[0,:].values
    df_list2.append(df2)

df = pd.concat(df_list2, axis=0, ignore_index=True)

In [ ]:
df

In [ ]:
import seaborn as sns
sns.set(font_scale=2)
sns.set_style("white")

In [ ]:
#plt.figure(figsize=(10,10))
lm = sns.lmplot(data=df, x='vx_drift', y='vx_top', hue='sigma_factor', col='fourier_radius', sharex=True, sharey=True, height=10, legend_out=False)
#plt.axis([-0.5, 0.5, -0.5, 0.5])
lm.axes[0,0].grid()
lm.axes[0,1].grid()

In [ ]:
p_top_0 = df[df['fourier_radius']==10.0].drop_duplicates('p_top_0').sort_values(by=['sigma_factor'])['p_top_0']
p_top_0

In [ ]:
lm = sns.lmplot(data=df[df['fourier_radius']==10.0], x='vx_top', y='vx_drift', hue='sigma_factor', height=10, legend_out=False)
lm.axes[0,0].grid()
leg = lm.axes.flat[0].get_legend()
leg.set_title(r'[$\sigma_m$: modifier to the stand. dev.]')
tempdf = df[df['fourier_radius']==10.0].drop_duplicates('p_top_0').sort_values(by=['sigma_factor'])
new_labels = [fr'$\sigma_m$ = {row["sigma_factor"]}; slope = {row["p_top_0"]:.2f}' for _, row in tempdf.iterrows()]
for t, l in zip(leg.texts, new_labels): t.set_text(l)
plt.title(r'$R_s$ = 8 px - Fourier filtering @ 1/(10 px)')

In [ ]:
for index, row in tempdf.iterrows():
    print(row['sigma_factor'])

In [ ]:
df.drop_duplicates('p_top_0').sort_values(by=['sigma_factor'])